In [23]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests 
from config import weather_api_key
import time
from datetime import datetime
url = 'https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=' + weather_api_key 

In [24]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [25]:
coordinates = list(lat_lngs)

In [26]:
# create a list for holding the cities
cities = []

# Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we'll add it to the cities list
    if city not in cities:
        cities.append(city)
        
len(cities)

576

In [27]:
# create an empty list to hold the weather data
city_data = []

# print the beginning of the logging
print("Beginning data retrieval       ")
print("-------------------------------")

# create counters
record_count = 1
set_count = 1

for i, city in enumerate(cities):

    # group cities in sets of 50 for logging purposes
    if(i % 50 == 0 and i >= 50):
            set_count += 1
            record_count = 1
            time.sleep(60)

    # create an endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ", "+")

    # log the URL, record and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        # Parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind= city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # append the city information into city_data list.
        city_weather_stats={"City": city.title(),
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_humidity,
                         "Cloudiness": city_clouds,
                         "Wind Speed": city_wind,
                         "Country": city_country,
                         "Date": city_date}
        city_data.append(city_weather_stats)

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
    pass

# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning data retrieval       
-------------------------------
Processing Record 1 of Set 1 | sitka
Processing Record 2 of Set 1 | bluff
Processing Record 3 of Set 1 | bubaque
Processing Record 4 of Set 1 | panguna
Processing Record 5 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 6 of Set 1 | saint john
Processing Record 7 of Set 1 | babanusah
City not found. Skipping...
Processing Record 8 of Set 1 | berlevag
Processing Record 9 of Set 1 | ilulissat
Processing Record 10 of Set 1 | ushuaia
Processing Record 11 of Set 1 | esperance
Processing Record 12 of Set 1 | santa teresa
Processing Record 13 of Set 1 | castro
Processing Record 14 of Set 1 | poronaysk
Processing Record 15 of Set 1 | khatanga
Processing Record 16 of Set 1 | banyo
Processing Record 17 of Set 1 | fevralsk
City not found. Skipping...
Processing Record 18 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 19 of Set 1 | yellowknife
Processing Record 20 of Set 1 | lithgow
Processin

In [28]:
len(city_data)

534

In [30]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sitka,57.0531,-135.3300,37.29,35,0,4.61,US,2022-04-13 16:20:02
1,Bluff,-46.6000,168.3333,44.80,83,81,13.18,NZ,2022-04-13 16:20:03
2,Bubaque,11.2833,-15.8333,75.04,73,100,15.43,GW,2022-04-13 16:20:04
3,Panguna,-6.3164,155.4848,66.70,94,46,2.26,PG,2022-04-13 16:20:04
4,Saint John,45.2727,-66.0677,56.43,32,40,19.57,CA,2022-04-13 16:20:05


In [32]:
new_column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']
city_data_df = city_data_df[new_column_order]
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Sitka,US,2022-04-13 16:20:02,57.0531,-135.3300,37.29,35,0,4.61
1,Bluff,NZ,2022-04-13 16:20:03,-46.6000,168.3333,44.80,83,81,13.18
2,Bubaque,GW,2022-04-13 16:20:04,11.2833,-15.8333,75.04,73,100,15.43
3,Panguna,PG,2022-04-13 16:20:04,-6.3164,155.4848,66.70,94,46,2.26
4,Saint John,CA,2022-04-13 16:20:05,45.2727,-66.0677,56.43,32,40,19.57
...,...,...,...,...,...,...,...,...,...
529,Simao,CN,2022-04-13 16:31:35,22.7886,100.9748,67.01,50,6,2.39
530,Mutoko,ZW,2022-04-13 16:35:48,-17.3970,32.2268,70.30,66,97,4.23
531,Sao Jose Da Coroa Grande,BR,2022-04-13 16:35:49,-8.8978,-35.1478,83.17,70,75,7.76
532,Guantanamo,CU,2022-04-13 16:35:49,20.1444,-75.2092,87.37,49,54,12.48


In [35]:
# create the output file (CSV)
output_data_file = "weather_data/cities.csv"

# export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")